In [2]:
graphics.off()  # clear all graphs
rm(list = ls()) # remove all files from your workspace

# IRS swap pricer
f_zero_pricer_IRS <- function(
    fixed_rate,                   # fixed rate
    vd.fixed_date, vd.float_date, # date for two legs
    vd.zero_date,  v.zero_rate,   # zero curve (dates, rates)
    d.spot_date,                  # spot date
    no_amt) {                     # nominal principal amount

    # Preprocessing
    # convert spot date from date(d) to numeric(n)
    n.spot_date <- as.numeric(d.spot_date)

    # Interpolation of zero curve
    vn.zero_date <- as.numeric(vd.zero_date)
    f_linear <- approxfun(vn.zero_date, v.zero_rate,
                          method="linear")
    vn.zero_date.inter <- n.spot_date:max(vn.zero_date)
    v.zero_rate.inter  <- f_linear(vn.zero_date)

    # number of CFs
    ni <- length(vd.fixed_date)
    nj <- length(vd.float_date)

    # output dataframe with CF dates and its interpolated zero
    df.fixed = data.frame(d.date = vd.fixed_date,
                          n.date = as.numeric(vd.fixed_date))
    df.float = data.frame(d.date = vd.float_date,
                          n.date = as.numeric(vd.float_date))

    # Fixed Leg
    # zero rate for discounting
    df.fixed$zero_DC = f_linear(as.numeric(df.fixed$d.date))

    # discount factor
    df.fixed$DF <- exp(-df.fixed$zero_DC *
                       (df.fixed$n.date - n.spot_date)/365)

    # tau, CF
    for(i in 1:ni) {

        ymd      <- df.fixed$d.date[i]
        ymd_prev <- df.fixed$d.date[i-1]
        if(i==1) ymd_prev <- d.spot_date

        d <- as.numeric(strftime(ymd, format = "%d"))
        m <- as.numeric(strftime(ymd, format = "%m"))
        y <- as.numeric(strftime(ymd, format = "%Y"))

        d_prev <- as.numeric(strftime(ymd_prev, format = "%d"))
        m_prev <- as.numeric(strftime(ymd_prev, format = "%m"))
        y_prev <- as.numeric(strftime(ymd_prev, format = "%Y"))

        # 30I/360
        tau <- (360*(y-y_prev) + 30*(m-m_prev) + (d-d_prev))/360

        # cash flow rate
        df.fixed$rate[i] <- fixed_rate

        # Cash flow at time ti
        df.fixed$CF[i] <- fixed_rate * tau * no_amt # day fraction
    }

    # Present value of CF
    df.fixed$PV = df.fixed$CF * df.fixed$DF

    # Floating Leg
    # zero rate for discounting
    df.float$zero_DC = f_linear(as.numeric(df.float$d.date))

    # discount factor
    df.float$DF <- exp(-df.float$zero_DC *
                       (df.float$n.date - n.spot_date)/365)

    # tau, forward rate, CF
    for(i in 1:nj) {

        date      <- df.float$n.date[i]
        date_prev <- df.float$n.date[i-1]

        DF        <- df.float$DF[i]
        DF_prev   <- df.float$DF[i-1]

        if(i==1) {
            date_prev <- n.spot_date
            DF_prev   <- 1
        }

        # ACT/360
        tau <- (date - date_prev)/360

        # forward rate
        fwd_rate <- (1/tau) * (DF_prev/DF - 1)

        # cash flow rate
        df.float$rate[i] <- fwd_rate

        # Cash flow amount at time ti
        df.float$CF[i] <- fwd_rate * tau * no_amt # day fraction
    }

    # Present value of CF
    df.float$PV = df.float$CF * df.float$DF

    return(sum(df.fixed$PV) - sum(df.float$PV))
}

# objective function to be minimized
objf <- function(
    v.unknown_swap_zero_rate, # unknown zero curve (rates)
    vn.unknown_swap_maty,     # unknown swap maturity
    v.swap_rate,              # fixed rate
    vd.fixed_date,            # date for fixed leg
    vd.float_date,            # date for float leg
    vd.zero_date_all,         # all dates for zero curve
    v.zero_rate_known,        # known zero curve (rates)
    d.spot_date,              # spot date
    no_amt) {                 # nominal principal amount

    # zero curve augmented with zero rates for swaps
    v.zero_rate_all <- c(v.zero_rate_known,  v.unknown_swap_zero_rate)

    v.swap_price <- NULL

    k <- 1
    for(i in vn.unknown_swap_maty) {

        # calculate IRS swap price
        swap_price <- f_zero_pricer_IRS(
            v.swap_rate[k],         # fixed rate,
            vd.fixed_date[1:(2*i)], # semi-annual date
            vd.float_date[1:(4*i)], # quarterly   date
            vd.zero_date_all,       # zero curve (dates)
            v.zero_rate_all,        # zero curve (rates)
            d.spot_date,            # spot date,
            no_amt)                 # nominal principal amount

        print(paste0("Swap Price at spot date = ", round(swap_price,6)))

        # concatenate swap prices
        v.swap_price <- c(v.swap_price, swap_price)
        k <- k + 1
    }

    return(sum(v.swap_price^2))
}

# Zero curve from Bloomberg as of 2021-06-30 until 5-year maturity
df.market <- data.frame(

    d.date = as.Date(c("2021-10-04","2021-12-15",
                       "2022-03-16","2022-06-15",
                       "2022-09-21","2022-12-21",
                       "2023-03-15","2023-07-03",
                       "2024-07-02","2025-07-02",
                       "2026-07-02")),

    # we use swap rate not zero rate.
    swap_rate= c(0.00145750000000000,
                 0.00139609870272047,
                 0.00203838571440434,
                 0.00197747863867587,
                 0.00266249271921742,
                 0.00359490949297661,
                 0.00512603194652204,
                 0.00328354999423027,
                 0.00571049988269806,
                 0.00793000012636185,
                 0.00964949995279312
    ),

    # zero rate is only used for comparison.
    zero_rate = c(0.00147746193495074,
                  0.00144337757980778,
                  0.00166389741542625,
                  0.00175294804717070,
                  0.00196071374597585,
                  0.00224582504806747,
                  0.00264462838911974,
                  0.00328408008984121,
                  0.00571530169527018,
                  0.00795496282359075,
                  0.00970003866673104
    )
)

# Libor Swap Specification

d.spot_date  <- as.Date("2021-07-02")    # spot date (date type)
n.spot_date  <- as.numeric(d.spot_date)  # spot date (numeric type)

no_amt     <- 10000000      # notional principal amount

# swap cash flow schedule from Bloomberg
lt.cf_date <- list(

    fixed = as.Date(c("2022-01-04","2022-07-05",
                      "2023-01-03","2023-07-03",
                      "2024-01-02","2024-07-02",
                      "2025-01-02","2025-07-02",
                      "2026-01-02","2026-07-02")),

    float = as.Date(c("2021-10-04","2022-01-04",
                      "2022-04-04","2022-07-05",
                      "2022-10-03","2023-01-03",
                      "2023-04-03","2023-07-03",
                      "2023-10-02","2024-01-02",
                      "2024-04-02","2024-07-02",
                      "2024-10-02","2025-01-02",
                      "2025-04-02","2025-07-02",
                      "2025-10-02","2026-01-02",
                      "2026-04-02","2026-07-02"))
)

# for bootstrapped zero curve
df.zero <- data.frame(
    d.date = df.market$d.date,
    n.date = as.numeric(df.market$d.date),
    tau    = as.numeric(df.market$d.date) - n.spot_date,
    taui   = as.numeric(df.market$d.date) - n.spot_date,
    swap_rate = df.market$swap_rate,
    zero_rate = rep(0,length(df.market$d.date)),
    DF        = rep(0,length(df.market$d.date)))

# tau(i) = t(i) – t(i-1)
df.zero$taui[2:nrow(df.zero)] <-
    df.zero$n.date[2:nrow(df.zero)] -
    df.zero$n.date[1:(nrow(df.zero)-1)]

# Bootstrapping – Deposit : row 1

# calculate discount factor for deposit
df.zero$DF[1] <- 1/(1+df.zero$swap_rate[1]*df.zero$tau[1]/360)

# convert DF to spot rate
df.zero$zero_rate[1] <- 365/df.zero$tau[1]*log(1/df.zero$DF[1])

df.zero

# Bootstrapping – Futures : rows from 2 to 7
# No convexity adjustment is made
for(i in 2:7) {

    # 1) discount factor from t(i-1) to t(i)
    df.zero$DF[i] <- 1/(1+df.zero$swap_rate[i]*df.zero$taui[i]/360)

    # 2) discount factor from spot date to t(i)
    df.zero$DF[i] <- df.zero$DF[i-1]*df.zero$DF[i]

    # 3) zero rate from discount factor
    df.zero$zero_rate[i] <- 365/df.zero$tau[i]*log(1/df.zero$DF[i])
}

df.zero_until_futures <- df.zero

# Bootstrapping – Swaps : rows from 8 to 11

# method 1 : Sequential Optimization for each observed swap maturity
# Bootstrapping zero rates sequentially using Brent minimization with known (already bootstrapped) zero rates
# initialization for fair comparison
df.zero <- df.zero_until_futures

for(i in 8:11) {

    # find one unknown zero rate for one swap maturity
    m <- optim(0.01, objf,
        control = list(abstol=10^(-20), reltol=10^(-20),
                       maxit=50000, trace=2),
        method = c("Brent"),
        lower = 0, upper = 0.1,                # for Brent
        vn.unknown_swap_maty = 2:(i-6),        # unknown zero maturity
        v.swap_rate = df.zero$swap_rate[8:i],  # observed swap rate
        vd.fixed_date = lt.cf_date$fixed,      # date for fixed leg
        vd.float_date = lt.cf_date$float,      # date for float leg
        vd.zero_date_all = df.zero$d.date[1:i],# all dates for zero curve
        v.zero_rate_known  = df.zero$zero_rate[1:(i-1)], # known zero rates
        d.spot_date = d.spot_date, no_amt = no_amt)

    # update this zero curve with the newly found zero rate
    df.zero$zero_rate[i] <- m$par

    # convert this new zero rate to discount factor
    df.zero$DF[i] <- exp(-df.zero$zero_rate[i]*df.zero$tau[i]/365)
}

df.zero_seq <- df.zero # output for sequential optimization

# method 2 : Global Optimization
# initialization for 2nd optimization for fair comparison
df.zero <- df.zero_until_futures

# find 4 unknown zero rates for each swap maturity
m <- optim(c(0.01, 0.01, 0.01, 0.01), objf,
    control = list(abstol=10^(-20), reltol=10^(-20),
                   maxit=50000, trace=2),
    method = c("Nelder-Mead"),
    vn.unknown_swap_maty = 2:5,             # unknown zero maturity
    v.swap_rate = df.zero$swap_rate[8:11],  # observed swap rate
    vd.fixed_date = lt.cf_date$fixed,       # date for fixed leg
    vd.float_date = lt.cf_date$float,       # date for float leg
    vd.zero_date_all = df.zero$d.date[1:11],# all dates for zero curve
    v.zero_rate_known  = df.zero$zero_rate[1:7], # known zero rates
    d.spot_date = d.spot_date, no_amt = no_amt)

# update this zero curve with the newly found 4 zero rates
df.zero$zero_rate[8:11] <- m$par

# convert this new zero rates to discount factors
df.zero$DF[8:11] <- exp(-df.zero$zero_rate[8:11] *
                        df.zero$tau[8:11]/365)

df.zero_glb <- df.zero # output for global optimization

# Comparison of two zero curves
df.output <- data.frame(date     = df.market$d.date,
                        zero_mkt = df.market$zero_rate,
                        zero_seq = df.zero_seq$zero_rate,
                        zero_glb = df.zero_glb$zero_rate)

# to avoid redundant expressions of df.output$ ….
df.output <- within(df.output, {
    diff_seq = zero_mkt - zero_seq;
    diff_glb = zero_mkt - zero_glb
})

print("Comparison with Bloomberg Zero Curve")
df.output


d.date,n.date,tau,taui,swap_rate,zero_rate,DF
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-10-04,18904,94,94,0.001457500,0.001477462,0.9996196
2021-12-15,18976,166,72,0.001396099,0.000000000,0.0000000
2022-03-16,19067,257,91,0.002038386,0.000000000,0.0000000
2022-06-15,19158,348,91,0.001977479,0.000000000,0.0000000
2022-09-21,19256,446,98,0.002662493,0.000000000,0.0000000
2022-12-21,19347,537,91,0.003594909,0.000000000,0.0000000
2023-03-15,19431,621,84,0.005126032,0.000000000,0.0000000
2023-07-03,19541,731,110,0.003283550,0.000000000,0.0000000
2024-07-02,19906,1096,365,0.005710500,0.000000000,0.0000000


[1] "Swap Price at spot date = -671996.80915"
[1] "Swap Price at spot date = -1100471.848658"
[1] "Swap Price at spot date = -396875.526937"
[1] "Swap Price at spot date = -222777.702347"
[1] "Swap Price at spot date = -113596.717366"
[1] "Swap Price at spot date = 8503.771307"
[1] "Swap Price at spot date = 1163.94866"
[1] "Swap Price at spot date = -55.21945"
[1] "Swap Price at spot date = 0.470104"
[1] "Swap Price at spot date = -0.003203"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = 0.000974"
[1] "Swap Price at spot date = -0.000977"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = -917193.16709"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = -1529880.048628"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = -516250.680291"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap Price at spot date = -259533.769911"
[1] "Swap Price at spot date = -1e-06"
[1] "Swap

date,zero_mkt,zero_seq,zero_glb,diff_glb,diff_seq
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-10-04,0.001477462,0.001477462,0.001477462,-1.084202e-18,-1.084202e-18
2021-12-15,0.001443378,0.001450496,0.001450496,-7.118815e-06,-7.118815e-06
2022-03-16,0.001663897,0.001668496,0.001668496,-4.598145e-06,-4.598145e-06
2022-06-15,0.001752948,0.001756344,0.001756344,-3.395756e-06,-3.395756e-06
2022-09-21,0.001960714,0.001963363,0.001963363,-2.649604e-06,-2.649604e-06
2022-12-21,0.002245825,0.002248026,0.002248026,-2.200602e-06,-2.200602e-06
2023-03-15,0.002644628,0.002646531,0.002646531,-1.902936e-06,-1.902936e-06
2023-07-03,0.003284080,0.003284072,0.003284072,8.045010e-09,8.044944e-09
2024-07-02,0.005715302,0.005715303,0.005715303,-1.505792e-09,-1.519486e-09
